<a href="https://colab.research.google.com/github/vaughnpaul/scholarshipai/blob/main/scholarshipai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
import os
!pip install pyairtable
from pyairtable import Table
from pyairtable import Api

# 1. Install pyairtable (if you haven't already):
!pip install pyairtable

# 2. Set Airtable API Key and Base ID as Colab Secrets:
#    - Go to Colab settings (gear icon) -> Secrets.
#    - Add two secrets:
#        - AIRTABLE_API_KEY (Your Airtable API key)
#        - AIRTABLE_BASE_ID (Your Airtable Base ID)

# 3. Retrieve secrets from environment variables:
from google.colab import userdata
AIRTABLE_API_KEY = userdata.get('AIRTABLE_API_KEY')
AIRTABLE_BASE_ID = userdata.get('AIRTABLE_BASE_ID')

if not AIRTABLE_API_KEY or not AIRTABLE_BASE_ID:
    raise ValueError("AIRTABLE_API_KEY and AIRTABLE_BASE_ID must be set as Colab Secrets.")

AIRTABLE_TABLE_NAME = "scholarship"  # Your Airtable table name
AIRTABLE_VIEW_ID = "viwQKJVMUyyZaCLzn" # Your Airtable View ID

# Initialize Airtable API object:
api = Api(AIRTABLE_API_KEY)

# Initialize Airtable table object using the Api object:
table = api.table(AIRTABLE_BASE_ID, AIRTABLE_TABLE_NAME, view=AIRTABLE_VIEW_ID) # Pass the view ID

def get_scholarship_data_from_airtable(table, question_field="question_text", name_field="scholarship_name"):
    """Retrieves question_text and scholarship_name from Airtable.

    Handles missing fields and returns a list of dictionaries.
    """
    try:
        records = table.all() # no need to specify the view here since it is already passed when creating the table object
        scholarship_data = []
        for record in records:
            fields = record.get("fields", {})
            question = fields.get(question_field)
            name = fields.get(name_field)

            # Append even if one field is missing, using empty strings as placeholders
            scholarship_data.append({
                "question_text": question if question else "",
                "scholarship_name": name if name else ""
            })

        return scholarship_data
    except Exception as e:
        print(f"Error retrieving data from Airtable: {e}")
        return []


essay_prompts = get_scholarship_data_from_airtable(table)

if essay_prompts:
    print("Essay prompts from Airtable:")
    for prompt in essay_prompts:
        print(prompt)

    try:
        import pandas as pd
        df = pd.DataFrame(essay_prompts)
        print("\nPandas DataFrame:")
        print(df)
    except ImportError:
        print("\nPandas is not installed. Install it with: !pip install pandas")
    except Exception as e:
        print(f"\nCould not create Pandas DataFrame: {e}")

else:
    print("No scholarship data found in Airtable.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 2.8 MB/s eta 0:00:00


TypeError: Api.table() got an unexpected keyword argument 'view'